In [ ]:
import polars as pl
import datetime as dt
import pandas as pd
df = pl.DataFrame(
    {
        "name": ["Alice Archer", "Ben Brown", "Chloe Cooper", "Daniel Donovan"],
        "birthdate": [
            dt.date(1997, 1, 10),
            dt.date(1985, 2, 15),
            dt.date(1983, 3, 22),
            dt.date(1981, 4, 30),
        ],
        "weight": [57.9, 72.5, 53.6, 83.1],  # (kg)
        "height": [1.56, 1.77, 1.65, 1.75],  # (m)
    }
)

# print(df)

df.write_csv("../data/cache/polars.csv")
# df_csv = pl.read_csv("../data/cache/polars.csv", try_parse_dates=True)

df_csv = pl.read_csv('../data/raw/us_options_opra/minute_aggs_v1/2025/07/2025-07-01.csv.gz')

print(df_csv.head())

shape: (4, 4)
┌────────────────┬────────────┬────────┬────────┐
│ name           ┆ birthdate  ┆ weight ┆ height │
│ ---            ┆ ---        ┆ ---    ┆ ---    │
│ str            ┆ date       ┆ f64    ┆ f64    │
╞════════════════╪════════════╪════════╪════════╡
│ Alice Archer   ┆ 1997-01-10 ┆ 57.9   ┆ 1.56   │
│ Ben Brown      ┆ 1985-02-15 ┆ 72.5   ┆ 1.77   │
│ Chloe Cooper   ┆ 1983-03-22 ┆ 53.6   ┆ 1.65   │
│ Daniel Donovan ┆ 1981-04-30 ┆ 83.1   ┆ 1.75   │
└────────────────┴────────────┴────────┴────────┘
shape: (5, 8)
┌────────────────────┬────────┬──────┬───────┬──────┬──────┬─────────────────────┬──────────────┐
│ ticker             ┆ volume ┆ open ┆ close ┆ high ┆ low  ┆ window_start        ┆ transactions │
│ ---                ┆ ---    ┆ ---  ┆ ---   ┆ ---  ┆ ---  ┆ ---                 ┆ ---          │
│ str                ┆ i64    ┆ f64  ┆ f64   ┆ f64  ┆ f64  ┆ i64                 ┆ i64          │
╞════════════════════╪════════╪══════╪═══════╪══════╪══════╪════════════════════

In [1]:
import polars as pl

data_dir = "../data/lake/us_stocks_sip/minute_aggs_v1/2025/07/*.parquet"

lf = pl.scan_parquet(data_dir)

# 转换时间戳，方便取最后一日
lf = lf.with_columns(
    pl.from_epoch("window_start", time_unit="ns").alias("datetime")
)

# 计算 VWAP (每个 ticker)
vwap = (
    lf.group_by("ticker")
      .agg((pl.col("close") * pl.col("volume")).sum() / pl.col("volume").sum())
      .rename({"close": "vwap"})
)

# 找到每个 ticker 当月最后的 close
last_close = (
    lf.sort("datetime")
      .group_by("ticker")
      .tail(1)
      .select(["ticker", "close"])
      .rename({"close": "last_close"})
)

# 合并结果，计算比率
result = (
    vwap.join(last_close, on="ticker")
        .with_columns((pl.col("last_close") / pl.col("vwap")).alias("ratio"))
        .sort("ratio", descending=True)
        .limit(1)
)

print(result.collect())


shape: (1, 4)
┌────────┬──────────┬────────────┬───────────┐
│ ticker ┆ vwap     ┆ last_close ┆ ratio     │
│ ---    ┆ ---      ┆ ---        ┆ ---       │
│ str    ┆ f64      ┆ f32        ┆ f64       │
╞════════╪══════════╪════════════╪═══════════╡
│ GVH    ┆ 0.133029 ┆ 5.56       ┆ 41.795326 │
└────────┴──────────┴────────────┴───────────┘


In [2]:
import duckdb

data_dir = "../data/lake/us_stocks_sip/minute_aggs_v1/2025/07/*.parquet"
con = duckdb.connect()

query = f"""
WITH vwap AS (
    SELECT 
        ticker,
        SUM(close * volume)::DOUBLE / NULLIF(SUM(volume),0) AS vwap
    FROM '{data_dir}'
    GROUP BY ticker
),
last_close AS (
    SELECT DISTINCT ON (ticker)
        ticker, close AS last_close
    FROM '{data_dir}'
    ORDER BY ticker, window_start DESC
)
SELECT 
    vwap.ticker,
    vwap.vwap,
    last_close.last_close,
    last_close.last_close / vwap.vwap AS ratio
FROM vwap
JOIN last_close USING (ticker)
ORDER BY ratio DESC
LIMIT 1;
"""

result = con.sql(query).df()
print(result)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

  ticker      vwap  last_close      ratio
0    GVH  0.133029        5.56  41.795326
